<a href="https://colab.research.google.com/github/roman-ruzica/hackprague2020_ganchors/blob/main/hackaton2020_data_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sklearn as sklearn
import statsmodels.api as sm
import calendar
from joblib import dump, load
import datetime

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
data = pd.read_csv("/content/drive/My Drive/CS_HACKATHON/hackprague_txs(1).csv", sep = ";")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (8,12,13) have mixed types.Specify dtype option on import or set low_memory=False.



In [ ]:
## create yearmonth column just in case
data['year'] = pd.DatetimeIndex(data['tx_date']).year
data['month'] = pd.DatetimeIndex(data['tx_date']).month
data['day'] = 1
data["yearmonth"] = pd.to_datetime(data[["year", "month", "day"]])

In [ ]:
## only take the full months which are not in the future

data2 = data[data['yearmonth'] > datetime.datetime(2020,4,15)]
data2 = data2[data['yearmonth'] < datetime.datetime(2020,10,20)]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [ ]:
data_small = data[["client_id", "region"]].groupby(["client_id"]).agg(pd.Series.mode)


In [ ]:
data_small2 = data_small.reset_index().rename({"region":"client_region"}, axis =1)

In [ ]:
data_small = data_small.reset_index()

In [ ]:
pop_table = pd.read_csv("/content/drive/My Drive/CS_HACKATHON/pop_table - Sheet1.csv", sep = ",")

In [ ]:
pop_table["region"] = pop_table["region"].astype("category")

In [ ]:
data_small["region"] = pd.Series(data_small["region"])

In [ ]:
data_small[:,["region"]]

In [ ]:
data_small.to_csv("data_small_mapping.csv", index = False)

In [ ]:
data_small_read = pd.read_csv("data_small_mapping.csv")

In [ ]:
town_mapping = data_small_read\
.merge(pop_table,how = 'inner', on = ["region"])

In [ ]:
data3 = pd.merge(data2, town_mapping, how = "inner", on = "client_id")

In [ ]:
data2["client_age"] = 2020 - data2["client_year_of_birth"]

In [ ]:
data3 = data3.rename({"region_y":"client_home_region"}, axis =  1)

In [ ]:
data3.to_csv("enriched_data2.csv")

In [ ]:
data3 = pd.read_csv("/content/drive/My Drive/CS_HACKATHON/enriched_data2.csv")

In [ ]:
client_table = data3.groupby(["client_id", "population", "client_home_region", "client_gender", "merchant_category", "client_year_of_birth"]).agg({"amount":"sum"}).reset_index()

client_table2 = client_table.groupby(["client_id", "client_gender", "population", "client_home_region", "client_year_of_birth"]).agg({"amount":"sum"})\
.rename({"amount":"tot_m_amount"}).reset_index()
client_table_tot = client_table.merge(client_table2,on = ["client_id", "population", "client_home_region", "client_gender", "client_year_of_birth"])
client_table_tot["client_home_region"] = client_table_tot["client_home_region"].astype('category')
client_table_tot["client_home_region_cat"] = client_table_tot["client_home_region"].cat.codes
client_table_tot.head()
client_table_tot[["gen_dummy", "gd2"]] = pd.get_dummies(client_table_tot["client_gender"])

In [ ]:
client_table_tot.to_csv("/content/drive/My Drive/CS_HACKATHON/model_data.csv")
#client_table_tot = pd.read_csv("/content/drive/My Drive/CS_HACKATHON/model_data2.csv")